In [1]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
import pandas as pd

dfs = {
    2019: pd.read_csv('final2019.csv'),
    2020: pd.read_csv('final2020.csv'),
    2021: pd.read_csv('final2021.csv'),
    2022: pd.read_csv('final2022.csv'),
    2023: pd.read_csv('final2023.csv'),
}

for year, df in dfs.items():
    print(f"--- Colonnes {year} ---")
    print(df.columns.tolist(), "\n")

/tmp/ipykernel_399563/1594908137.py:4: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  2019: pd.read_csv('final2019.csv'),
/tmp/ipykernel_399563/1594908137.py:5: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  2020: pd.read_csv('final2020.csv'),
/tmp/ipykernel_399563/1594908137.py:6: DtypeWarning: Columns (7,8,18,20,22,28,30,32,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  2021: pd.read_csv('final2021.csv'),
/tmp/ipykernel_399563/1594908137.py:7: DtypeWarning: Columns (7,8,20,22,24,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  2022: pd.read_csv('final2022.csv'),


--- Colonnes 2019 ---
['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longitude', '

/tmp/ipykernel_399563/1594908137.py:8: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  2023: pd.read_csv('final2023.csv'),


In [5]:
import pandas as pd

# 1) Concaténation
df = pd.concat(dfs.values(), ignore_index=True)
print("Shape après concaténation :", df.shape)

# 2) Date → datetime + extraction de l'année
df['date_mutation'] = pd.to_datetime(df['date_mutation'], dayfirst=True, errors='coerce')
df['year'] = df['date_mutation'].dt.year

Shape après concaténation : (543908, 51)


/tmp/ipykernel_399563/2325805555.py:8: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['date_mutation'] = pd.to_datetime(df['date_mutation'], dayfirst=True, errors='coerce')


In [6]:
# 1) Nettoyage de la variable DPE
df['dpe_clean'] = df['Etiquette_DPE'].str.upper().str.strip()

# 2) Création de l'indicatrice de traitement
# Le groupe de traitement est constitué des logements avec une étiquette DPE F ou G.
df['treat'] = df['dpe_clean'].isin(['F', 'G']).astype(int)

# 3) Définir l'indicateur 'post' qui vaut 1 si la date de mutation est postérieure à la date clé, sinon 0.
df['post_exact'] = (df['date_mutation'] >= '2021-08-24').astype(int)

# 4) Création de l'indicatrice DID qui est le produit de 'treat' et 'post_exact'
# Cette variable identifie les observations du groupe traité après la période de l'intervention.
df['did'] = df['treat'] * df['post_exact']

# 5) Création de l'indicatrice de groupe de contrôle (0 pour A, B, C, D, 1 pour F, G)
# Le groupe de contrôle est constitué des logements avec une étiquette DPE A, B, C ou D
df['control'] = df['dpe_clean'].isin(['A', 'B', 'C', 'D']).astype(int)



In [8]:
# 1) liste des contrôles
controls = [
    'surface_reelle_bati',  # Surface réelle bâtie
    'nombre_pieces_principales',  # Nombre de pièces principales
    'Année_construction',  # Année de construction
    'Type_bâtiment',  # Indicateur de valeurs manquantes dans l'année de construction
]

# 2) construction de la liste complète des variables du modèle
vars_model = ['valeur_fonciere', 'treat', 'post_exact', 'did', 'code_departement'] + controls

# 3) comptage des missing
missing = df[vars_model].isna().sum().rename('n_missing')
total   = df[vars_model].shape[0]
pct     = (missing / total * 100).round(2).rename('pct_missing')
print(pd.concat([missing, pct], axis=1))

# 4) création du jeu clean (drop NA)
df_mod = df.dropna(subset=vars_model)
print(f"\nObservations avant : {total:,}  –  après dropna : {len(df_mod):,}")


                           n_missing  pct_missing
valeur_fonciere                   29         0.01
treat                              0         0.00
post_exact                         0         0.00
did                                0         0.00
code_departement                   0         0.00
surface_reelle_bati                0         0.00
nombre_pieces_principales          0         0.00
Année_construction             99451        18.28
Type_bâtiment                      0         0.00

Observations avant : 543,908  –  après dropna : 444,429


In [10]:

# 1) Indicateur de missing + imputation pour l'année de construction
df['year_const_miss'] = df['Année_construction'].isna().astype(int)
median_year = df['Année_construction'].median()
df['Année_construction_imp'] = df['Année_construction'].fillna(median_year)

# 2) Nouvelle liste de contrôles (sans surface_terrain, avec l’imputation)
controls2 = [
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'Type_bâtiment',
    'Année_construction_imp',
    'year_const_miss'
]

vars_model2 = ['valeur_fonciere', 'treat', 'post_exact', 'did', 'code_departement'] + controls2

# 3) Nouveau diagnostic des missing
missing2 = df[vars_model2].isna().sum().rename('n_missing')
pct2     = (missing2 / df.shape[0] * 100).round(2).rename('pct_missing')
print(pd.concat([missing2, pct2], axis=1))

# 4) Jeu « clean » après dropna
df_mod2 = df.dropna(subset=vars_model2)
print(f"\nObservations avant : {df.shape[0]:,}  –  après dropna : {len(df_mod2):,}")

                           n_missing  pct_missing
valeur_fonciere                   29         0.01
treat                              0         0.00
post_exact                         0         0.00
did                                0         0.00
code_departement                   0         0.00
surface_reelle_bati                0         0.00
nombre_pieces_principales          0         0.00
Type_bâtiment                      0         0.00
Année_construction_imp             0         0.00
year_const_miss                    0         0.00

Observations avant : 543,908  –  après dropna : 543,879


In [14]:
import statsmodels.formula.api as smf

# Formule du modèle Diff-in-Diff avec variables de contrôle
formula = (
    'np.log(valeur_fonciere) ~ treat + post_exact + did + '
    'C(code_type_local) + surface_reelle_bati + Type_bâtiment + '
    'Année_construction_imp + year_const_miss'
)

# Régression avec erreurs standards robustes clusterisées par département
model = smf.ols(formula, data=df).fit(
    cov_type='cluster',
    cov_kwds={'groups': df['code_departement']}
)

# Résultats
print(model.summary())



TypeError: '<' not supported between instances of 'int' and 'str'